# BACKTESTING ANALISYS

In [370]:
import pandas as pd
import numpy as np
import pandas_datareader.data as pdr
from datetime import datetime, date, timedelta
import math
import warnings
warnings.filterwarnings("ignore")
from matplotlib import pyplot as plt

import statistics as s

import seaborn as sns
sns.set()
sns.set_theme()

## LECTURA DE DATOS

In [371]:
# Lectura de datos
start = datetime(1900,1,1)
end = datetime.now()
data = pdr.get_data_yahoo('BTC-USD', start, end, interval='d')

data = data[['Adj Close']].copy()
data= data.rename(columns={'Adj Close': 'btc_price'})
data['btc_return'] = data['btc_price'].pct_change()
data['day'] = data.index.day
data['weekday'] = data.index.dayofweek
data['month'] = data.index.month
data['year'] = data.index.year
data 



,btc_price,btc_return,day,weekday,month,year
Date,,,,,,
2014-09-17,457.334015,NaN,17,2,9,2014
2014-09-18,424.440002,-0.071926,18,3,9,2014
2014-09-19,394.795990,-0.069843,19,4,9,2014
2014-09-20,408.903992,0.035735,20,5,9,2014
2014-09-21,398.821014,-0.024659,21,6,9,2014
...,...,...,...,...,...,...
2021-11-24,56280.425781,-0.022384,24,2,11,2021
2021-11-25,57274.679688,0.017666,25,3,11,2021
2021-11-26,53569.765625,-0.064687,26,4,11,2021


In [372]:
data.to_csv('datos_btc.csv')

## FUNCIONES

## Cálculo de retornos atípicos

In [373]:
def statistics(df):
    
    '''Retorna un DataFrame con las estadísticas resumen de la variable "btc_return" para identificar cuando se considera un retorno atípico.'''

    statistics_returns = df['btc_return'].describe()
    statistics_returns['IQR'] = statistics_returns['75%'] - statistics_returns['25%']
    statistics_returns['AI'] = statistics_returns['25%'] - 1.5 * statistics_returns['IQR']
    statistics_returns['AS'] = statistics_returns['75%'] + 1.5 * statistics_returns['IQR']
    statistics_returns['AextI'] = statistics_returns['25%'] - 3 * statistics_returns['IQR']
    statistics_returns['AextS'] = statistics_returns['75%'] + 3 * statistics_returns['IQR']
    statistics_returns['std_AI'] = statistics_returns['AI'] / statistics_returns['std']
    statistics_returns['std_AextI'] = statistics_returns['AextI'] / statistics_returns['std']
    statistics_returns['std_AextS'] = statistics_returns['AextS'] / statistics_returns['std']
    print(statistics_returns)

    return statistics_returns
   

In [374]:
data1 = data[(data['year'] >= 2021)] 
atipicos = statistics(data1)


count        332.000000
mean           0.002988
std            0.042894
min           -0.137661
25%           -0.021902
50%            0.002371
75%            0.026347
max            0.187465
IQR            0.048249
AI            -0.094275
AS             0.098721
AextI         -0.166648
AextS          0.171094
std_AI        -2.197854
std_AextI     -3.885112
std_AextS      3.988757
Name: btc_return, dtype: float64


## Calculo del CAGR

In [375]:

estrategies = ['_E1', '_E2', '_E3', '_E4']

def cagr(df):
    '''
    Retorna un dataframe con el cálculo del CAGR por estrategia de compra, realizando el cálculo en función de los usdt que entran y los que salen.
    '''
    cagr_results = []
    for e in estrategies: 
        df = df.copy()
        #df = df.sort_index(ascending=True)
        Begining_value = df['usdt_invested'+e].sum()
        Ending_Value = df['btc_amount'+e].sum() * df['btc_price'][-1]  
        total_ret = Ending_Value / Begining_value

        start = datetime.utcfromtimestamp(df.index.values[0].astype('O')/1e9)
        end = datetime.utcfromtimestamp(df.index.values[-1].astype('O')/1e9)
        period_years = (end - start).days / 365.25
        CAGR = round((total_ret)**(1/period_years)-1, 4)
        cagr_results.append(CAGR)
        #print(f'Estrategia{e}: ', round(CAGR,3))
    cagr_results = pd.DataFrame({'cagr': cagr_results}, index=['DCA_simple', 'DCA_variable', 'DCA_variable_caídas', 'DCA_variable_atipicos'])
    return cagr_results

## Estrategias DCA

In [376]:

def DCA_simple(df, usdt):
   '''
   Retorna un dataframe con la cantidad de usdt invertidos según la frecuencia de compra
   '''
   for x in df.index:
      row = df.loc[x]
      df.at[x,'usdt_invested_E1'] = usdt
    
   df['btc_amount_E1'] = df['usdt_invested_E1']/df['btc_price']
   return df.head(20)

In [377]:

def DCA_variable(df, usdt, var_usdt):
   '''
   Retorna un dataframe con la cantidad de usdt invertidos (según la variación del retorno) y la cantidad de btc obtenidos.
   '''
   std_return = data['btc_return'].std()
   for x in df.index:
      row = df.loc[x]
      if(row['btc_return'] < atipicos['AI']):
         df.at[x,'usdt_invested_E2'] = usdt*(1+var_usdt)
      elif(row['btc_return'] > atipicos['AS']):    
         df.at[x,'usdt_invested_E2'] = usdt*(1-var_usdt)
      else:
         df.at[x, 'usdt_invested_E2'] = usdt
   df['btc_amount_E2'] = df['usdt_invested_E2']/df['btc_price']
   return df.head(20)

In [378]:

def DCA_variable_caida(df, usdt):
   '''
   Retorna un dataframe con la cantidad de usdt invertidos según la variación del retorno y la cantidad de btc obtenidos
   '''
   for x in df.index:
      row = df.loc[x]
      if(row['btc_return'] < -0.0):
         df.at[x,'usdt_invested_E3'] = 2*usdt*(1+np.abs(row['btc_return']))
      else:
         df.at[x, 'usdt_invested_E3'] = usdt
   df['btc_amount_E3'] = df['usdt_invested_E3']/df['btc_price']
   return df.head(20)

In [379]:
def DCA_variable_atipicos(df, usdt):
   
   '''
   Retorna un dataframe con la cantidad de usdt invertidos según el número de desviaciones estandar que cae el retorno
   '''
  
   for x in df.index:
      row = df.loc[x]
      if(row['btc_return'] > atipicos['AI']) & (row['btc_return'] < atipicos['AS']):
         df.at[x,'usdt_invested_E4'] = usdt
      elif(row['btc_return'] < atipicos['AI']):    
         df.at[x,'usdt_invested_E4'] = usdt + usdt*np.abs(atipicos['AI'])
      elif(row['btc_return'] < atipicos['AextI']):
         df.at[x,'usdt_invested_E4'] = usdt + usdt*np.abs(atipicos['AextI']) 
      #elif(row['btc_return'] > (Atipico_ext_inf*std_return)):
        # df.at[x,'usdt_invested_E4'] = 0
      elif(row['btc_return'] > atipicos['AS']):
         df.at[x,'usdt_invested_E4'] = 0
      else:
         df.at[x, 'usdt_invested_E4'] = usdt
   df['btc_amount_E4'] = df['usdt_invested_E4']/df['btc_price']
   return df.head(20)

In [380]:
# DataFrame semanal
usdt = 50
df = data.copy()
df = df[(df['year'] >= 2017) & (df['weekday']==0)] 
#df = df[df['weekday']==0] 
df['btc_return'] = df['btc_price'].pct_change() 
DCA_simple(df, usdt)
DCA_variable(df, usdt,1)
DCA_variable_caida(df,usdt)
DCA_variable_atipicos(df,usdt)


df.head(10)

,btc_price,btc_return,day,weekday,month,year,usdt_invested_E1,btc_amount_E1,usdt_invested_E2,btc_amount_E2,usdt_invested_E3,btc_amount_E3,usdt_invested_E4,btc_amount_E4
Date,,,,,,,,,,,,,,
2017-01-02,1021.750000,NaN,2,0,1,2017,50.0,0.048936,50.0,0.048936,50.000000,0.048936,50.000000,0.048936
2017-01-09,902.828003,-0.116391,9,0,1,2017,50.0,0.055382,100.0,0.110763,111.639050,0.123655,54.713741,0.060603
2017-01-16,831.533997,-0.078967,16,0,1,2017,50.0,0.060130,50.0,0.060130,107.896743,0.129756,50.000000,0.060130
2017-01-23,921.012024,0.107606,23,0,1,2017,50.0,0.054288,0.0,0.000000,50.000000,0.054288,0.000000,0.000000
2017-01-30,920.382019,-0.000684,30,0,1,2017,50.0,0.054325,50.0,0.054325,100.068404,0.108725,50.000000,0.054325
2017-02-06,1038.150024,0.127956,6,0,2,2017,50.0,0.048163,0.0,0.000000,50.000000,0.048163,0.000000,0.000000
2017-02-13,990.642029,-0.045762,13,0,2,2017,50.0,0.050472,50.0,0.050472,104.576217,0.105564,50.000000,0.050472
2017-02-20,1079.979980,0.090182,20,0,2,2017,50.0,0.046297,50.0,0.046297,50.000000,0.046297,50.000000,0.046297
2017-02-27,1179.969971,0.092585,27,0,2,2017,50.0,0.042374,50.0,0.042374,50.000000,0.042374,50.000000,0.042374


In [381]:
# Cálculo del CAGR por estrategia
cagr_btc = round(cagr(df),3)
cagr_btc


,cagr
DCA_simple,0.626
DCA_variable,0.634
DCA_variable_caídas,0.624
DCA_variable_atipicos,0.636


In [382]:

def calculate_return(df):
    # retorno E1: DCA Simple
    total_btc_amount_E1 = df['btc_amount_E1'].sum()
    total_usdt_invested_E1 = df['usdt_invested_E1'].sum()
    total_usdt_obtained_E1 = round(total_btc_amount_E1*df['btc_price'][-1],3)
    total_return_E1 = round((total_usdt_obtained_E1/total_usdt_invested_E1-1)*100,2)
    
    # retorno E2: DCA Variable
    total_btc_amount_E2 = df['btc_amount_E2'].sum()
    total_usdt_invested_E2 = df['usdt_invested_E2'].sum()
    total_usdt_obtained_E2 = round(total_btc_amount_E2*df['btc_price'][-1],3)
    total_return_E2 = round((total_usdt_obtained_E2/total_usdt_invested_E2-1)*100,2)


   # retorno E3: DCA Variable en caídas
    total_btc_amount_E3 = df['btc_amount_E3'].sum()
    total_usdt_invested_E3 = df['usdt_invested_E3'].sum()
    total_usdt_obtained_E3 = round(total_btc_amount_E3*df['btc_price'][-1],3)
    total_return_E3 = round((total_usdt_obtained_E3/total_usdt_invested_E3-1)*100,2)

    # retorno E4: DCA Variable en caídas2
    total_btc_amount_E4 = df['btc_amount_E4'].sum()
    total_usdt_invested_E4 = df['usdt_invested_E4'].sum()
    total_usdt_obtained_E4 = round(total_btc_amount_E4*df['btc_price'][-1],3)
    total_return_E4 = round((total_usdt_obtained_E4/total_usdt_invested_E4-1)*100,2)

    results = pd.DataFrame({'total_usdt_invested': [total_usdt_invested_E1, total_usdt_invested_E2, total_usdt_invested_E3, total_usdt_invested_E4],
                             'total_usdt_obtained': [total_usdt_obtained_E1, total_usdt_obtained_E2, total_usdt_obtained_E3, total_usdt_obtained_E4],
                             'total_return': [total_return_E1, total_return_E2, total_return_E3, total_return_E4]}, 
                             index=['DCA_simple', 'DCA_variable', 'DCA_variable_caídas', 'DCA_variable_atipicos'])
    return results
  

In [383]:
#retorno total por compra semanal
calculate_return(df)

,total_usdt_invested,total_usdt_obtained,total_return
DCA_simple,12800.000000,138886.022,985.05
DCA_variable,11350.000000,126155.358,1011.50
DCA_variable_caídas,19499.942928,210727.614,980.66
DCA_variable_atipicos,10172.557278,113708.617,1017.80


In [384]:
# DataFrame semanal
years = data['year'].unique().tolist()
usdt = 50

for i in years:

    df = data.copy()
    df = df[(df['year'] == i) & (df['weekday']==0)] 
    df['btc_return'] = df['btc_price'].pct_change() 
    DCA_simple(df, usdt)
    DCA_variable(df, usdt,1)
    DCA_variable_caida(df,usdt)
    DCA_variable_atipicos(df,usdt)
    retornos = calculate_return(df)
    print('_____________________________________________________________________________________')
    print(f'Año: {i}', retornos)
   

_____________________________________________________________________________________
Año: 2014                        total_usdt_invested  total_usdt_obtained  total_return
DCA_simple                      750.000000              652.005        -13.07
DCA_variable                    700.000000              616.717        -11.90
DCA_variable_caídas            1307.214092             1153.058        -11.79
DCA_variable_atipicos           654.713741              573.820        -12.36
_____________________________________________________________________________________
Año: 2015                        total_usdt_invested  total_usdt_obtained  total_return
DCA_simple                     2600.000000             4157.149         59.89
DCA_variable                   2600.000000             4286.453         64.86
DCA_variable_caídas            3741.135994             6031.459         61.22
DCA_variable_atipicos          2373.568707             3883.359         63.61
____________________________